In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
import numpy as np
from ktrain import text

input = pd.read_csv('../data/complete_ktrain.csv')

(x_train, y_train), (x_val, y_val), preprocessing = text.texts_from_df(train_df=input, text_column='sentence',
    label_columns=['joy', 'trust', 'fear', 'surprise', 'sadness', 'disgust', 'anger', 'anticipation', 'neutral'],
                        val_pct=0.2, max_features=1000, maxlen=75)
print(x_train.shape)
print(y_train.shape)
print(x_train)

# generate balanced weights for training
from sklearn.utils import class_weight
def generate_balanced_weights(y_train):
    y_labels = [y.argmax() for y in y_train]
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_labels), y_labels)
    weight_dict = {}
    for key in range(len(class_weights)):
        weight_dict[key] = class_weights[key]
    return weight_dict

class_weight_dict = generate_balanced_weights(y_train)
print(class_weight_dict)

# model
from keras.layers import Input, Embedding, LSTM, Dense, Bidirectional
from keras.models import Model

max_length = 75
max_words = 1000
features = 200
classes = 9
input_1 = Input(shape=(max_length,))
embed_1 = Embedding(input_dim=(max_words - 1), output_dim=features, input_length=max_length)(input_1)
bi_lstm_1 = Bidirectional(LSTM(units=32, activation='tanh', dropout=0.2, return_sequences=True))(embed_1)
bi_lstm_2 = Bidirectional(LSTM(units=32, activation='tanh', dropout=0.2, return_sequences=True))(bi_lstm_1)
bi_lstm_3 = Bidirectional(LSTM(units=16, activation='tanh', dropout=0.2, return_sequences=False))(bi_lstm_2)
softmax_1 = Dense(units=classes, activation='softmax')(bi_lstm_3)

model = Model(inputs=input_1, outputs=softmax_1)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print(model.summary())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

using Keras version: 2.2.4


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


ModuleNotFoundError: No module named 'stellargraph'

In [13]:
history = model.fit(x=x_train, y=y_train, validation_data=(x_val, y_val), batch_size=16, epochs=3, class_weight=class_weight_dict)

Train on 720 samples, validate on 180 samples
Epoch 1/3
720/720 [==============================] - 26s 36ms/step - loss: 2.2035 - acc: 0.1694 - val_loss: 2.1954 - val_acc: 0.1389
Epoch 2/3
720/720 [==============================] - 19s 27ms/step - loss: 2.1883 - acc: 0.1486 - val_loss: 2.1857 - val_acc: 0.2222
Epoch 3/3
720/720 [==============================] - 19s 26ms/step - loss: 2.1243 - acc: 0.2514 - val_loss: 2.1334 - val_acc: 0.2222


In [14]:
import ktrain

In [15]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_val, y_val))
predictor = ktrain.get_predictor(learner.model, preproc=preprocessing)

In [16]:
test = pd.read_csv('../data/test_set_ktrain.csv')
sentences = test['sentence'].values
for s in sentences:
    print(s)
    p = predictor.predict(s, return_proba=True)
    print(p)
    predictor.explain(s)
    break

ZiffleRick  you wanted to know what I wanted from the forum
[0.1464651  0.12408463 0.09495255 0.06504862 0.16994403 0.08673582
 0.05949852 0.06154834 0.19172236]
